In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import warnings
import xgboost as xgb
import lightgbm as lgb
from scipy.stats import skew
from scipy import stats
from scipy.stats.stats import pearsonr
from scipy.stats import norm
from collections import Counter

import sklearn
import sklearn.model_selection
import sklearn.ensemble
import sklearn.metrics
import sklearn.model_selection

from sklearn.model_selection import KFold, cross_val_score, train_test_split

In [2]:
train = pd.read_csv('Train.csv.zip', compression='zip')
test = pd.read_csv('Test.csv.zip', compression='zip')
# train.columns

In [3]:
# Save the 'Id' column
train_ID = train['id']
test_ID = test['id']

# Now drop the 'Id' column since it's unnecessary for the prediction process.
train.drop("id", axis = 1, inplace = True)
test.drop("id", axis = 1, inplace = True)

In [4]:
# Combining Datasets
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.price.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['price'], axis=1, inplace=True)
print("Train data size is : {}".format(train.shape))
print("Test data size is : {}".format(test.shape))
print("Combined dataset size is : {}".format(all_data.shape))

Train data size is : (100000, 24)
Test data size is : (100000, 23)
Combined dataset size is : (200000, 23)


/home/axreldable/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [5]:
# настройка параметров
all_data[['build_tech']] = all_data[['build_tech']].replace(np.nan, 0.5)
all_data[['metro_dist']] = all_data[['metro_dist']].replace(np.nan, 15.0)
all_data[['g_lift']] = all_data[['g_lift']].replace(np.nan, 0.5)

In [6]:
all_data.drop("date", axis = 1, inplace = True)
all_data.drop("kw1", axis = 1, inplace = True)
all_data.drop("kw2", axis = 1, inplace = True)
all_data.drop("kw3", axis = 1, inplace = True)
all_data.drop("kw4", axis = 1, inplace = True)
all_data.drop("kw5", axis = 1, inplace = True)
all_data.drop("kw6", axis = 1, inplace = True)
all_data.drop("kw7", axis = 1, inplace = True)
all_data.drop("kw8", axis = 1, inplace = True)
all_data.drop("kw9", axis = 1, inplace = True)
all_data.drop("kw10", axis = 1, inplace = True)
all_data.drop("kw11", axis = 1, inplace = True)
all_data.drop("kw12", axis = 1, inplace = True)
all_data.drop("kw13", axis = 1, inplace = True)

In [7]:
# Check if there are any missing values left
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

,Missing Ratio


In [8]:
all_data
all_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 9 columns):
area          200000 non-null int64
balcon        200000 non-null int64
build_tech    200000 non-null float64
floor         200000 non-null int64
g_lift        200000 non-null float64
metro_dist    200000 non-null float64
n_photos      200000 non-null int64
rooms         200000 non-null int64
street_id     200000 non-null int64
dtypes: float64(3), int64(6)
memory usage: 13.7 MB


In [9]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [10]:
X, Xt, y, yt = sklearn.model_selection.train_test_split(train, y_train, test_size=0.3, random_state=0)

In [11]:
model_xgb = xgb.XGBRegressor(max_depth=13, n_estimators=5000, random_state =7) # 861814.873034375

# XGBRegressor max_depth=6 n_estimators=100: 1675995.2668916667
# XGBRegressor max_depth=6 n_estimators=5000: 959617.0378790039
# XGBRegressor max_depth=6 n_estimators=10000: 911419.2695312988
# XGBRegressor max_depth=6 n_estimators=15000: 902241.6986599203

# XGBRegressor max_depth=6 n_estimators=100: 1282773.6694083333 заполненный g_lift
# XGBRegressor max_depth=6 n_estimators=100: 1284701.1225666667 пустой g_lift
# XGBRegressor max_depth=6 n_estimators=100: 1294787.9690958334 с датой как dummies
# XGBRegressor max_depth=6 n_estimators=100: 1285825.3127229167 с включенным kw10

# XGBRegressor max_depth=11 n_estimators=100: 1031950.7442583333
# XGBRegressor max_depth=12 n_estimators=100: 1011165.3294583333
# XGBRegressor max_depth=13 n_estimators=100: 1009005.9016822916
# XGBRegressor max_depth=14 n_estimators=100: 1027168.0148854167
# XGBRegressor max_depth=24 n_estimators=100: 1270361.10094375

# XGBRegressor max_depth=13 n_estimators=5000: 864670.0302248698
# XGBRegressor max_depth=13 n_estimators=10000: 864670.2538933593

In [12]:
# model_xgb.fit(X, y)

# pred = model_xgb.predict(Xt)

# score = sklearn.metrics.mean_absolute_error(yt, pred)

# print('XGBRegressor max_depth=13 n_estimators=10000: %s' % (score))

# GradientBoostingRegressor

In [13]:
# Cross-validation with k-folds
# n_folds = 3

# def rmsle_cv(model):
#     kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
#     rmse= -cross_val_score(model, train.values, y_train, scoring="neg_mean_absolute_error", cv = kf)
#     return(rmse)

In [14]:
# model_gbm = sklearn.ensemble.GradientBoostingRegressor(max_depth=3, n_estimators=100, random_state=0) # 1570901.5855248524
# model_gbm = sklearn.ensemble.GradientBoostingRegressor(max_depth=10, n_estimators=100, random_state=0) # 1025566.5043463386
# model_gbm = sklearn.ensemble.GradientBoostingRegressor(max_depth=11, n_estimators=100, random_state=0) # 998213.6576657181
# model_gbm = sklearn.ensemble.GradientBoostingRegressor(max_depth=12, n_estimators=100, random_state=0) # 1006102.1627678034
# model_gbm = sklearn.ensemble.GradientBoostingRegressor(max_depth=13, n_estimators=100, random_state=0) # 1009792.6467483348
# model_gbm = sklearn.ensemble.GradientBoostingRegressor(max_depth=15, n_estimators=100, random_state=0) # 11090817.7737429522

# model_gbm = sklearn.ensemble.GradientBoostingRegressor(max_depth=11, n_estimators=10000, random_state=0) # too long
# model_gbm = sklearn.ensemble.GradientBoostingRegressor(max_depth=3, n_estimators=10000, random_state=0) # 675804.6089045412
# model_gbm = sklearn.ensemble.GradientBoostingRegressor(max_depth=5, n_estimators=10000, random_state=0) # 698730.959269384

# k-fold
# model_gbm = sklearn.ensemble.GradientBoostingRegressor(max_depth=5, n_estimators=100, random_state=0) # 1370084.9505

model_gbm = sklearn.ensemble.GradientBoostingRegressor(max_depth=3, n_estimators=17500, random_state=0) # 670921.9636994784

In [15]:
# model_gbm.fit(X, y)

# pred = model_gbm.predict(Xt)

# score = sklearn.metrics.mean_absolute_error(yt, pred)

# print('Score Gradient Boosting max_depth=3, n_estimators=100: %s' % (score))

In [16]:
# score = rmsle_cv(model_gbm)
# print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [17]:
model_xgb.fit(train, y_train)

pred = model_xgb.predict(train)
score = sklearn.metrics.mean_absolute_error(y_train, pred)
print('XGBRegressor: %s' % (score))

XGBRegressor: 1697.9468903125


In [18]:
# model_gbm.fit(train, y_train)

# pred = model_gbm.predict(train)
# score = sklearn.metrics.mean_absolute_error(y_train, pred)
# print('GradientBoostingRegressor: %s' % (score))

In [19]:
# pred_rez = 0.2 * model_xgb.predict(test) + 0.8 * model_gbm.predict(test)

In [20]:
pred_rez = model_xgb.predict(test)

In [21]:
rez_test = test

In [22]:
type(test_ID)

pandas.core.series.Series

In [23]:
rez_test['price'] = pred_rez
rez_test['id'] = test_ID.values

rez_test[['id', 'price']].to_csv('sub.csv', index=False)

/home/axreldable/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/axreldable/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
